In [36]:
#coding=utf-8
import os
import chardet
import re 
import jieba
from glob import glob

def check_contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

def srt_process():
    if not os.path.exists('decode/'):
        os.makedirs('decode/')
    if not os.path.exists('seg/'):
        os.makedirs('seg/')

    for filename in glob('*.srt'):
        with open(filename, 'rb') as file:
            filename = filename.split('.')
            title = filename[0]
            contents = file.read()
            detect = chardet.detect(contents)#detect the encoding
            try: 
                lines = contents.decode(detect['encoding']).encode("utf8").split(b'\r\n')
                output = b''
                for l in lines:
                    output += l + b'\n'
                with open('decode/'+title+'.txt', 'wb') as fout:
                    fout.write(output)
                print('success  ----- {:16} --- {}'.format(detect['encoding'],title))
            except: 
                print('fail     ----- {:16} --- {}'.format(detect['encoding'],title))

            with open('decode/'+title+'.txt', 'r') as f:
                seg_out = ''
                for line in f:
                    if check_contain_chinese(line):
                        line = re.sub(r'[^\w]','',line) 
                        seg_list = jieba.cut(line)
                        if seg_list:
                            line_out = ''
                            for s in seg_list:
                                if check_contain_chinese(s):
                                    line_out += s + '/'
                            seg_out += line_out + '\n'
            with open('seg/'+title+'.txt', 'w') as f2:
                f2.write(seg_out)
                
def main():
    srt_process()
    
if __name__ == "__main__":
    main()

success  ----- UTF-8-SIG        --- About Time
success  ----- UTF-8-SIG        --- 2001: A Space Odyssey
success  ----- UTF-8-SIG        --- A Clockwork Orange
success  ----- Big5             --- Aliens
fail     ----- GB2312           --- Alien
success  ----- UTF-8-SIG        --- Avatar
